# SMR model (CMSR) v1
- Date: 23/01/2023
- Based on: https://www.kns.org/files/pre_paper/44/20A-079-김경원.pdf

Questions
1. Composition of Hastelloy N? Is it wo or ao? http://moltensalt.org/references/static/downloads/pdf/ORNL-TM-4189.pdf
1. Temperatures of fuel, moderator, cladding?

In [1]:
# Fuel properties
FUEL_TEMP = 900 # K
FUEL_ENRICHMENT = 4.0 # wo%
FUEL_R = 5 # cm

# Moderator properties
MODERATOR_TEMP = 900 # K

# Cladding properties
CLADDING_TEMP = 900 # K
CLADDING_R = 5 # cm

# Lattice properties
LATTICE_RINGS = 9
LATTICE_PITCH = 12 # cm

# Containment properties
CONTAINMENT_R = 120 # cm
CONTAINMENT_THICKNESS = 5 # cm

# Core properties
CORE_HEIGHT = 200 # cm

In [2]:
import openmc
import numpy as np
import matplotlib.pyplot as plt
import os

# @@@@@@@@@@@@@@@@@@@@@@ Materials @@@@@@@@@@@@@@@@@@@@@@
# Fuel
material_fuel = openmc.Material(1, "NaF-KF-UF4", temperature=FUEL_TEMP)
material_fuel.add_elements_from_formula('NaFKFUF4', enrichment=FUEL_ENRICHMENT)
material_fuel.set_density('g/cm3', 4.261)

# Cladding
material_cladding = openmc.Material(2, "HastelloyN", temperature=CLADDING_TEMP)
material_cladding.add_element('Ni', 0.72, 'wo')
material_cladding.add_element('Mo', 0.16, 'wo')
material_cladding.add_element('Cr', 0.07, 'wo')
material_cladding.add_element('Cr', 0.05, 'wo')
material_cladding.set_density('g/cm3', 8.86)

# Moderator
material_moderator = openmc.Material(3, "NaOH", temperature=MODERATOR_TEMP)
material_moderator.add_elements_from_formula('NaOH')
material_moderator.set_density('g/cm3', 1.637)

# Create Materials list and export to XML
mats = openmc.Materials([material_fuel, material_cladding, material_moderator])
mats.export_to_xml()
# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



# @@@@@@@@@@@@@@@@@@@@@@ Geometry @@@@@@@@@@@@@@@@@@@@@@
# Cylinders
r_pin = openmc.ZCylinder(r=FUEL_R)
r_cladding = openmc.ZCylinder(r=CLADDING_R)

# Regions
region_fuel = -r_pin
region_cladding = +r_pin & -r_cladding
region_moderator = +r_cladding

# Cells
cell_fuel = openmc.Cell(fill=material_fuel, region=region_fuel)
cell_cladding = openmc.Cell(fill=material_cladding, region=region_cladding)
cell_moderator = openmc.Cell(fill=material_moderator, region=region_moderator)

# Universe
universe_pin = openmc.Universe(cells=(cell_fuel, cell_cladding, cell_moderator))

# Outer universe
cell_all_moderator = openmc.Cell(fill=material_moderator)
universe_outer = openmc.Universe(cells=(cell_all_moderator,))
# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@



# @@@@@@@@@@@@@@@@@@@@@@ Lattice @@@@@@@@@@@@@@@@@@@@@@
lattice = openmc.HexLattice()
lattice.center = (0., 0.)
lattice.pitch = (LATTICE_PITCH,)
lattice.outer = universe_outer

lattice.universes = [[universe_pin]*(6*i + (i==0)*1) for i in range(LATTICE_RINGS)[::-1]]

r_containment_inner = openmc.ZCylinder(r=CONTAINMENT_R)
r_containment_outer = openmc.ZCylinder(r=CONTAINMENT_R+CONTAINMENT_THICKNESS, boundary_type='vacuum')

region_containment = +r_containment_inner & -r_containment_outer

cell_containment = openmc.Cell(fill=material_cladding, region=region_containment)
cell_main = openmc.Cell(fill=lattice, region=-r_containment_inner)
geometry = openmc.Geometry([cell_containment, cell_main])
geometry.export_to_xml()
# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [3]:
# print(lattice.show_indices(num_rings=LATTICE_RINGS))
# print(lattice)

In [4]:
# plot = openmc.Plot.from_geometry(geometry)
# plot.color_by = 'material'

# plot.to_ipython_image()

In [5]:
# outer_surface = openmc.ZCylinder(r=50, boundary_type='vacuum')
# main_cell = openmc.Cell(fill=lattice, region=-outer_surface)
# geometry = openmc.Geometry([main_cell])
# geometry.export_to_xml()